# Mask Classification

Every Convolutional block contains 2 conv layers (with size doubled block by block) and 1 maxpool layer with size 2x2 and stride 2.<br>
This model take 0.63 accuracy on test set.

In [2]:
import os
import numpy as np 
import tensorflow as tf 
import pandas

SEED = 1234

tf.random.set_seed(SEED)

cwd = os.getcwd();

### Data Augmentation
simple transformation as zoom, flip (horizontal and vertical), rotation and shear.<br>
From the dataset, the 75% is used for training set with all the trasformation and 25% for validation set


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


apply_data_augmentation = True

if apply_data_augmentation: #if data augmentation is enabled, create the generator
     train_data_gen = ImageDataGenerator(rotation_range=10,
                                        zoom_range=0.1,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        shear_range = 0.2, #added for TL
                                        validation_split = 0.25,
                                        rescale=1./255)                                       
else: #rescale only the image
     train_data_gen = ImageDataGenerator(rescale = 1./255)                                       

#rescale only on validation dataset and test dataset
valid_data_gen = ImageDataGenerator(rescale = 1./255, validation_split= 0.25)

test_data_gen = ImageDataGenerator(rescale = 1./255)


In [4]:
#width and height of images

img_w = 256
img_h = 256

num_classes = 3

classes = ["NO PERSON", "ALL THE PEOPLE", "SOMEONE"]

bs = 16 #batch size

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Loading data 
import json 

#setting dirs
dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir = os.path.join(dataset_dir, "training")
validation_dir = training_dir


#reading json file
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)


dataframe = pandas.DataFrame(dic.items())

dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)

dataframe["class"] = dataframe["class"].astype(str)

#shuffle data before loading and splitting

dataframe = dataframe.sample(frac = 1) 

train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               color_mode='rgb',
                                               subset='training',
                                               shuffle=True,
                                               seed=SEED) 

                                                                                        

valid_gen = valid_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               color_mode='rgb',
                                               subset='validation',
                                               shuffle=False,
                                               seed=SEED)   


 
                                                                                                                                                                                                

Found 4211 validated image filenames belonging to 3 classes.
Found 1403 validated image filenames belonging to 3 classes.


In [6]:
#Creating Dataset objects


train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()


valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()     


In [22]:
# Creating model 

model = tf.keras.Sequential()

#start size of the filter
start_f = 12

#depth of the feature extraction part
depth = 7

    

    # Features extraction
for i in range(depth):
    if i == 0:
         input_shape = [img_h, img_w, 3]
    else:
         input_shape=[None]

    # Conv block: Conv2D -> Activation -> Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    start_f *= 2
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())                                
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2)))
    
    
    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 256, 256, 12)      48        
_________________________________________________________________
re_lu_56 (ReLU)              (None, 256, 256, 12)      0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 256, 256, 24)      2616      
_________________________________________________________________
re_lu_57 (ReLU)              (None, 256, 256, 24)      0         
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 128, 128, 24)      0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 128, 128, 24)      600       
_________________________________________________________________
re_lu_58 (ReLU)              (None, 128, 128, 24)     

In [47]:
# Load the model
load_model = True
if load_model:
    #latest = tf.train.latest_checkpoint(os.path.join("C:\\Users\\Giovanni\\Desktop\\Università\\Magistrale\\ANN\\AN2DL-homeworks\\image-classification\\classification_experiments\\CNN_Nov21_11-58-00\\ckpts", ""))
    model.load_weights(os.path.join("C:\\Users\\Giovanni\\Desktop\\Università\\Magistrale\\ANN\\AN2DL-homeworks\\image-classification\\classification_experiments\\CNN_Nov21_11-58-00\\ckpts\\", "cp_03.ckpt"), by_name=False)


In [48]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

if load_model == True:
    model.evaluate(valid_dataset.take(len(valid_gen)))

88/88 [==============================] - 11s 121ms/step - loss: 0.7865 - accuracy: 0.6244


[0.7864541411399841, 0.6243763566017151]

In [14]:
from datetime import datetime



exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks.append(es_callback)

In [15]:
model.fit(x=train_dataset,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/30
264/264 [==============================] - 193s 731ms/step - loss: 1.0986 - accuracy: 0.3372 - val_loss: 1.0986 - val_accuracy: 0.3264
Epoch 2/30
 81/264 [========>.....................] - ETA: 1:38 - loss: 1.0985 - accuracy: 0.3264

KeyboardInterrupt: 

In [35]:
# Test Dataset 
# Useful if you want to see images with predictions  

#from PIL import Image
#image_filenames = next(os.walk('MaskDataset/test'))[2]

#results = {}
#for image_name in image_filenames:
#   img = Image.open('MaskDataset/test/'+image_name).convert('RGB')
#   img = img.resize((img_w,img_h))
#   img_array = np.array(img)
#   img_array = np.expand_dims(img_array, 0) 
#   img_array = tf.cast(img_array, tf.float32) / 255.
#   prediction = model.predict(img_array)
#   img
#   classes[np.argmax(prediction)]
   


In [49]:
#Creating CSV

import os
from datetime import datetime
from PIL import Image

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


image_filenames = next(os.walk('MaskDataset/test'))[2]

results = {}
for image_name in image_filenames:
   img = Image.open('MaskDataset/test/'+image_name).convert('RGB')
   img = img.resize((img_w,img_h))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 
   img_array = tf.cast(img_array, tf.float32) / 255.
   prediction = model.predict(img_array)
   results[image_name] = np.argmax(prediction)

create_csv(results)            